In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [5]:
%cd drive/MyDrive

/content/drive/MyDrive


In [8]:
%cd Chatbot

/content/drive/MyDrive/Chatbot


In [10]:
!ls

config.json  Llama_chat.ipynb  main.ipynb  requirements.txt  Untitled11.ipynb


In [11]:
# Install dependencies
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 68.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [ ]:
!pip install -U bitsandbytes

In [19]:
import json
import torch
from huggingface_hub import login
from transformers import (AutoModelForCausalLM,
                         AutoTokenizer,
                         BitsAndBytesConfig,
                         pipeline)

In [22]:
config_data = json.load(open("config.json"))
HF_TOKEN = config_data["HF_TOKEN"]
login(HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [23]:
# Load the model and tokenizer from Hugging Face
model_name = "meta-llama/Llama-2-13b-chat-hf"

In [24]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
#       load_in_8bit=True
)

In [25]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          use_auth_token = HF_TOKEN)

tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:796: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [26]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             torch_dtype=torch.float16,
                                             quantization_config=bnb_config,
                                             token=HF_TOKEN)

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [27]:
# Load the pipeline for text generation
text_generator = pipeline("text-generation",
                          model=model,
                          tokenizer=tokenizer,
                          max_new_tokens=512,)

In [34]:
# Chatbot function
def chat():
    print("Chatbot: Hello! How can I assist you today?")

    while True:
        user_input = input("You: ")

        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Chatbot: Goodbye!")
            break

        # Indented this line to be inside the while loop
        response = text_generator(user_input, max_new_tokens=200)[0]['generated_text']
        print(f"Chatbot: {response.strip()}\n")

In [35]:
chat()

Chatbot: Hello! How can I assist you today?
You: Taj Mahal
Chatbot: Taj Mahal: The White Marble Wonder of India
The Taj Mahal, located in Agra, India, is one of the most famous and iconic landmarks in the world. This stunning white marble mausoleum was built in the 17th century by Mughal Emperor Shah Jahan in memory of his beloved wife, Mumtaz Mahal, who died during childbirth. The Taj Mahal is considered one of the greatest examples of Mughal architecture, which blended Indian, Persian, and Islamic styles, and it is a UNESCO World Heritage Site.

The Taj Mahal is made of white marble, with intricate inlay work of precious stones, including jasper, jade, and turquoise. The mausoleum is surrounded by beautiful gardens and a large reflecting pool, which adds to its beauty and tranquility. The Taj Mahal

You: When was it built?
Chatbot: When was it built?

The current building of the church was built in 1875, but the history of the church dates back to the 12th century.

You: When was taj